## 项目信息

In [189]:
project_name = '堰桥友汇2'  # 项目名称
target_date = '20240509'  # 目标日期
target_packs = []  # 目标托盘
for i in range(1, 9):
    target_packs.append(i)
packs_df_dict = {}  # 托盘数据

## 原始数据

In [190]:
import os

import pandas as pd

# 文件路径
target_dir = f"../temp/{project_name}/{target_date}"
for target_pack in target_packs:
    bms_file = os.path.join(target_dir, f"base1_pack{target_pack}_bms1.csv")
    # 读取bms
    df = pd.read_csv(bms_file)
    columns = ['time', 'Energy', 'Rsen']
    for i in range(1, 17):
        columns.append(f'v{i}')
    df = df[columns]  # 筛选列
    df['time'] = target_date + ' ' + df['time']
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    # 保存
    packs_df_dict[target_pack] = df
print(f"共{len(packs_df_dict)}个托盘数据")

共8个托盘数据


## 计算电量

In [191]:

for target_pack in target_packs:
    # 每隔n条取一条
    packs_df_dict[target_pack] = packs_df_dict[target_pack].iloc[::10, :]
    # 筛选
    packs_df_dict[target_pack] = packs_df_dict[target_pack][packs_df_dict[target_pack]['Energy'] > 0]
    packs_df_dict[target_pack] = packs_df_dict[target_pack][packs_df_dict[target_pack]['Rsen'] > 1000] # 2.0版本过滤
    df = packs_df_dict[target_pack]
    # 计算
    df["Ah"] = df['Energy'] * 3.3 * 100 * 100000 * 100 / 4095 / df['Rsen'] / 3600 / 20 / 10 / 100 # 2.0
    # df["Ah"] = df['Energy'] / 30.23 / 3600 / 20 # 4.0
    df['Wh'] = df['Ah'] * 51.2
    df['SOC'] = df['Ah'] / df['Ah'].max() * 100
packs_df_dict[target_packs[0]]

,Energy,Rsen,v1,v2,v3,v4,v5,v6,v7,v8,...,v10,v11,v12,v13,v14,v15,v16,Ah,Wh,SOC
time,,,,,,,,,,,,,,,,,,,,,
2024-05-09 00:00:43,40045,5886,3.084,3.091,3.089,3.092,3.084,3.046,3.083,3.059,...,3.041,3.054,3.073,3.075,3.065,3.030,3.002,0.076147,3.898751,0.025979
2024-05-09 00:02:33,196892,5886,3.094,3.096,3.099,3.102,3.096,3.055,3.092,3.070,...,3.050,3.065,3.084,3.088,3.078,3.038,3.015,0.374400,19.169258,0.127733
2024-05-09 00:04:23,351177,5886,3.099,3.104,3.105,3.105,3.100,3.062,3.096,3.076,...,3.059,3.071,3.091,3.092,3.084,3.049,3.020,0.667780,34.190331,0.227825
2024-05-09 00:06:13,506929,5886,3.104,3.109,3.109,3.110,3.109,3.070,3.100,3.084,...,3.067,3.078,3.096,3.097,3.089,3.055,3.031,0.963950,49.354230,0.328868
2024-05-09 00:08:03,1068003,5886,3.142,3.146,3.149,3.155,3.149,3.110,3.136,3.131,...,3.104,3.118,3.138,3.136,3.123,3.094,3.075,2.030859,103.979976,0.692863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-09 21:14:30,7755092,5886,3.110,3.113,3.105,3.100,3.105,3.086,3.110,3.086,...,3.096,3.102,3.107,3.112,3.110,3.091,3.057,14.746679,755.029977,5.031086
2024-05-09 21:16:21,6708815,5886,3.092,3.097,3.096,3.084,3.089,3.065,3.094,3.062,...,3.076,3.084,3.086,3.096,3.094,3.070,3.034,12.757133,653.165228,4.352318
2024-05-09 21:18:11,5696434,5886,3.070,3.078,3.075,3.067,3.067,3.039,3.071,3.034,...,3.046,3.054,3.062,3.073,3.068,3.042,3.001,10.832042,554.600569,3.695540


## 绘图

In [204]:
import plotly.graph_objects as go
import random
import numpy as np

start_time = '17:10'
end_time = '23:59'
fig = go.Figure()
fig.update_layout(title=f'{project_name} {target_date} {start_time}-{end_time}')
fig.update_xaxes(title_text='Ah')
fig.update_yaxes(title_text='V')

for target_pack in target_packs:
    df = packs_df_dict[target_pack].between_time(start_time, end_time)  # 筛选时间
    random_color = f'#{random.randint(0, 0xffffff):06x}'
    for i in range(1, 17):
        Ah = df['Ah']
        V = df[f'v{i}']
        dV_dAh = np.diff(V) / np.diff(Ah)
        Ah_mid = (Ah[:-1] + Ah[1:]) / 2  # 中间点的Ah值
            
        # 筛选出一阶微分不为零的点
        non_zero_indices = np.nonzero(dV_dAh)[0]
        Ah_mid_non_zero = Ah_mid[non_zero_indices]
        dV_dAh_non_zero = dV_dAh[non_zero_indices]

        # 计算二阶微分
        if len(dV_dAh_non_zero) > 1:
            d2V_dAh2 = np.diff(dV_dAh_non_zero) / np.diff(Ah_mid_non_zero)
            Ah_mid_non_zero_mid = (Ah_mid_non_zero[:-1] + Ah_mid_non_zero[1:]) / 2  # 中间点的Ah值
                
        # 添加一阶微分图
        #fig.add_trace(go.Scatter(x=Ah_mid_non_zero, y=dV_dAh_non_zero, mode='markers',name=f'p{target_pack}_c{i}_dV/dAh', marker=dict(color=random_color, size=3)))
        # 添加二阶微分图
        #fig.add_trace(go.Scatter(x=Ah_mid_non_zero_mid, y=d2V_dAh2, mode='lines+markers',name=f'p{target_pack}_c{i}_d2V/dAh2', marker=dict(color=random_color, size=3, symbol='cross')))
        # x轴为Ah y轴为电压
        fig.add_trace(go.Scatter(x=df['Ah'], y=df[f'v{i}'], mode='markers',name=f'p{target_pack}_c{i}', marker=dict(color=random_color, size=3)))
fig.show()

C:\Users\29814\AppData\Local\Temp\ipykernel_68624\3029073726.py:23: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\29814\AppData\Local\Temp\ipykernel_68624\3029073726.py:18: RuntimeWarning:

divide by zero encountered in divide

C:\Users\29814\AppData\Local\Temp\ipykernel_68624\3029073726.py:18: RuntimeWarning:

invalid value encountered in divide

